## Mirroring Learning and MARL TRPO

### Part-01: MARL TRPO

Using PG methods in MARL issues offers poor convergence property. 

*Centralised Training with Decentralized Execution(CTDE)* method has a joint function for each agent. This method allows a straightforward extension of single-agent PG theorms to multi-agent scenarios. 

*IPP and MAPPP* methods are designed for agents that share the same action space and policy parameters, which suffer from an exponentially-worse suboptimal outcome. If applied in a non-parameters sharing way, it still lacks the essential theorietical property of trust region learning. 

Key findings: 

1. Multi-agent advantage decomposition lemma; 
2. Sequential policy update scheme; 

Task: fully-cooperative setting where all agents share the same reward function, aiming to maximize the expected total reward. 

Previous *MAPPO*: 

1. if parameter sharing, need the action space is equal and lead to suboptimal solution. 
2. if not paramter sharing. Agents' local imporvements in performance can jointly lead to a worse outcome. 

Based on the two lammas: 

$$ \mathbf{A}_{\pi}^{i_{1:m}} = \sum_{j=1}^{m} \mathbf{A}_{\pi}^{i_j}\left( s, \mathbf{a}^{i_{1:j-1}}, a_{i_j} \right) $$

$$ \mathcal{J}(\bar{\pi}) \geq \mathcal{J}(\pi) + \sum_{m=1}^{n} \left[ L_{\pi}^{i_{i:m}} - CD_{KL}^{max}(\pi^{i_m}, \bar{\pi}^{i_m})\right] $$

**these two lamma could lead out that: based on the update of each subset of agents, we could get the total Reward increased.**

#### Practical Algorithms

Parameterise each agent's policy $\pi_i$ by $\theta_i$, forms a joint policy $\mathbf{\pi}$ by $\theta =\left( \theta_1, \theta_2, ..., \theta_n \right)$

1. HATRPO

Similar to TRPO, one can ease the KL-divergence penalty $D_{KL}^{max}\left( \pi_{\theta_{k}^{i_m}}^{i_m}, \pi_{\theta^{i_m}}^{i_m} \right)$ by replacing it with expected-KL divergence constraint $\mathbf{\mathbb{E}}_{s \sim \rho_{\pi_{\theta_k}}} \left[ D_{KL} \left( \pi_{\theta_{k}^{i_m}}^{i_m}(\cdot | s), \pi_{\theta^{i_m}}^{i_m}(\cdot | s)  \right) \right] \leq \delta$

And, we can use a linear approximation to close the constrint. 

By the proof,  $\mathbf{E} \left[ A_{\pi}^{i_m} \left( s, \mathbf{a}^{i_{1:m-1}}, a^{i_m} \right) \right]  = \mathbf{E} \left[\left( \frac{\hat{\pi}^{i_m}(a^{i_m} | s)} {\pi^{i_m}(a^{i_m} | s)} - 1\right)\frac{\bar{\pi}^{i_{1:m-1}}(a^{1:m-1} | s)} {\pi^{i_{1:m-1}}(a^{1:m-1} | s)} A_{\mathbf{\pi}}(s, \mathbf{a})\right]$

2. HAPPO

We set the $\mathbf{M}^{i_{1:m}} = \frac{\bar{\pi}^{i_{1:m-1}}(a^{1:m-1} | s)} {\pi^{i_{1:m-1}}(a^{1:m-1} | s)}$

The HATROP can follow the idea of PPO which was achieved by making agent $i_m$ choose by policy parameter $\theta_{k+1}^{i_m}$ which maximize the clipping objective of: 

$$ \mathbf{E} \left[ min\left(  \frac{\pi_{\theta^{i_m}}^{i_m}(a^{i_m} | s)} {\pi_{\theta^{i_m}_{k}}^{i_m}(a^{i_m} | s)} M^{i_{1:m}}(s|a), clip\left( \frac{\pi_{\theta^{i_m}}^{i_m}(a^{i_m} | s)} {\pi_{\theta^{i_m}_{k}}^{i_m}(a^{i_m} | s)}, 1 \pm \epsilon \right)\right)M^{i_{1:m}}(s|a)\right]$$

#### Conclusion

The algorithms attain theoretically-justifed monotonical improvement property. By update the subset policy, we get the better and better global policy. 

### Part-02: Mirroring Learning

In this paper, which combine the TRPO, PPO, and Gradient Iteration methods to a unify theorietical proof framework which provides monotonic imporvement and optimal-policy convergence guarantee. 

1. Dirft Function

Drift function maps states and policies to a real space, and the following conditions are met: 

1. $\mathcal{D}_{\pi}(\bar{\pi} | s) \geq \mathcal{D}_{\pi}({\pi} | s) = 0$
2. $\mathcal{D}_{\pi}(\bar{\pi} | s) $ has zero gradient w.r.t ${\pi}(\cdot | s) $ evaluated at ${\bar{\pi}}(\cdot | s) = {\pi}(\cdot | s)$

2. Neighborhood operator 

We say that $\mathcal{N}: \prod \rightarrow \mathbb{P}(\prod)$, $ \mathbb{P}(\prod)$ is the power set of $\prod$, is neighborhood operator, if: 

1. it is a continuous map
2. Every $mathcal{N}(\pi)$ is a compact set
3. There exists a matrix $\mathcal{X}: \prod \times \prod \rightarrow \mathbb{R}$, s.t $\forall \pi \in \prod, \exists \zeta, s.t \mathcal{X}(\pi, \bar{\pi}) \leq \zeta$ implies $\bar{\pi} \in mathcal{N}(\pi)$

3. Mirror Operator

Mirrow operator transforms the value function of $\pi$ into the following functional of $\bar{\pi}$: 

$$ \left[  \mathcal{M}_{\mathcal{D}}^{\bar{\pi}} V_{\pi}(s) \right] = \mathbb{E}_{a \sim \bar{\pi}} \left[  Q_{\pi}(s, a) - \frac{v_{\bar{\pi}}^{\pi}(s)} {\beta_{\pi}(s)} \mathcal{D}_{\pi}(\bar{\pi} | s) \right]   $$

By the lamma: 

$\mathcal{M}_{\mathcal{D}}^{\pi_{new}} V_{\pi}(s)  \geq \mathcal{M}_{\mathcal{D}}^{\pi_{old}} V_{\pi}(s), \forall s \in \mathcal{S} \Rightarrow V_{\pi_{new}}(s) \geq V_{\pi_{old}}(s)$ 


If we update the policy by: 

$$ \pi_{new}  = argmax_{\bar{\pi}} \in \mathcal{N}(\pi_{old}) \mathbb{E}_{s \sim \beta_{\pi}} \left[  \left[ \mathcal{M}_{\mathcal{D}}^{\bar{\pi}} V_{\pi}(s) \right] \right] $$

We could using lamma 2 make out: 

$\mathcal{M}_{\mathcal{D}}^{\pi_{new}} V_{\pi}(s)  \geq \mathcal{M}_{\mathcal{D}}^{\pi_{old}} V_{\pi}(s), \forall s \in \mathcal{S} $


Thus, get better policy $\pi_{new}$

In order to get the Expectation, we could approximate by sampling with the "unbiased" estimator: 

$$  \frac{1} {|\mathcal{B}|} \sum_{s, a \in \mathcal{B}}\left[  \frac{\bar{\pi}(a | s)} {\pi_{old}(a | s)} Q_{\pi_{old}} (s, a)- \frac{v_{\pi_{old}}^{\bar{\pi}}(s)} {\beta_{\pi_{old}}(s)} \mathcal{D}_{\pi_{old}}(\bar{\pi} | s)  \right]$$

Therefore, we get the final result: 

Let $\pi_{0} \in \prod$, and the sequence of policies $(\pi_{n})_{n=0}^{\infty}$ be obtained by mirror learning induced by $\mathcal{D}^{v}, \mathcal{N}, \beta_{\pi}$:

1. Attain the strict monotonic improvement property: 
$$ \eta(\pi_{n+1}) \geq \eta(\pi_{n}) + \mathbb{E}_{s \sim d} \left[ \frac{v_{\pi_{n}}^{\pi_{n+1}}(s)} {\beta_{\pi_{n}}(s)} \mathcal{D}_{\pi_{n}}(\pi_{n+1} | s)   \right] $$
2. Their value functions converage to optimal one; 
$$ \lim_{n \rightarrow \infty} V_{\pi_{n}} = V^{*}$$
3. Their expected returns converage to the optimal return; 
$$ \lim_{n \rightarrow \infty} \eta_{\pi_{n}} = \eta^{*}$$


And baesd on the Theorem, the **General Policy Imporvement, Trust Region Learning, TRPO, PPO** all could fit in this framework. 